In [1]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')

omopcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x55565294fe00>


In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

In [3]:
df_VISIT_OCCURRENCE = pd.read_sql('select * from `VISIT_OCCURRENCE`', con=conn)

/tmp/ipykernel_530363/2903905030.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_VISIT_OCCURRENCE = pd.read_sql('select * from `VISIT_OCCURRENCE`', con=conn)


In [4]:
print('Length - VISIT_OCCURRENCE: ', len(df_VISIT_OCCURRENCE))

Length - VISIT_OCCURRENCE:  0


In [5]:
print(df_VISIT_OCCURRENCE.head())

Empty DataFrame
Columns: [visit_occurrence_id, person_id, visit_concept_id, visit_start_datetime, visit_end_datetime, visit_type_concept_id, provider_id, care_site_id, visit_source_value, visit_source_concept_id, admitted_from_concept_id, admitted_from_source_value, discharge_to_concept_id, discharge_to_source_value, preceding_visit_occurrence_id]
Index: []


In [6]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

mimicivconn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='mimiciv')

mimicivcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x55565294fe00>


In [7]:
df_patients_mimiciv = pd.read_sql('select subject_id, anchor_year, anchor_year_group  from patients', con=mimicivconn)

/tmp/ipykernel_530363/850515151.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_patients_mimiciv = pd.read_sql('select subject_id, anchor_year, anchor_year_group  from patients', con=mimicivconn)


In [8]:
ddf_patients_mimiciv = dd.from_pandas(df_patients_mimiciv).compute()

In [9]:
df_patients_mimiciv = ''

In [10]:
print(ddf_patients_mimiciv.head())

   subject_id  anchor_year anchor_year_group
0    10000032         2180       2014 - 2016
1    10000048         2126       2008 - 2010
2    10000068         2160       2008 - 2010
3    10000084         2160       2017 - 2019
4    10000102         2136       2008 - 2010


In [11]:
df_admissions_mimiciv = pd.read_sql('select * from admissions', con=mimicivconn)
print(df_admissions_mimiciv.head()) 

/tmp/ipykernel_530363/3412243236.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_admissions_mimiciv = pd.read_sql('select * from admissions', con=mimicivconn)


   subject_id   hadm_id           admittime           dischtime deathtime  \
0    10467237  20000019 2159-03-20 21:08:00 2159-03-23 16:54:00       NaT   
1    16925328  20000024 2151-05-25 21:44:00 2151-05-26 18:36:00       NaT   
2    19430048  20000034 2174-05-22 19:13:00 2174-05-24 17:27:00       NaT   
3    18910522  20000041 2143-09-03 07:15:00 2143-09-06 13:50:00       NaT   
4    11146739  20000057 2190-01-15 17:07:00 2190-01-18 16:05:00       NaT   

                admission_type admit_provider_id  admission_location  \
0                     EW EMER.            P25U4M      EMERGENCY ROOM   
1                     EW EMER.            P54Q2B      EMERGENCY ROOM   
2           DIRECT OBSERVATION            P30SZF  PHYSICIAN REFERRAL   
3  SURGICAL SAME DAY ADMISSION            P48VXK  PHYSICIAN REFERRAL   
4               EU OBSERVATION            P77SO2      EMERGENCY ROOM   

         discharge_location insurance language marital_status  \
0                      HOME     Other  

In [12]:
ddf_admissions_mimiciv = dd.from_pandas(df_admissions_mimiciv).compute()

In [13]:
df_admissions_mimiciv = ''

In [14]:
print(ddf_admissions_mimiciv)

        subject_id   hadm_id           admittime           dischtime  \
0         10467237  20000019 2159-03-20 21:08:00 2159-03-23 16:54:00   
1         16925328  20000024 2151-05-25 21:44:00 2151-05-26 18:36:00   
2         19430048  20000034 2174-05-22 19:13:00 2174-05-24 17:27:00   
3         18910522  20000041 2143-09-03 07:15:00 2143-09-06 13:50:00   
4         11146739  20000057 2190-01-15 17:07:00 2190-01-18 16:05:00   
...            ...       ...                 ...                 ...   
431226    10382924  29999723 2170-08-09 18:56:00 2170-08-10 18:30:00   
431227    11326722  29999745 2160-07-05 22:24:00 2160-07-12 12:50:00   
431228    12133002  29999809 2172-09-24 21:27:00 2172-09-30 17:04:00   
431229    16572164  29999828 2180-10-27 07:15:00 2180-10-30 17:25:00   
431230    16549335  29999928 2165-07-28 17:22:00 2165-07-31 11:53:00   

       deathtime               admission_type admit_provider_id  \
0            NaT                     EW EMER.            P25U4M   
1

In [15]:
ddf_adm_pat_merged = ddf_admissions_mimiciv.merge(ddf_patients_mimiciv, how='inner', on='subject_id')
print(ddf_adm_pat_merged.head()) 

   subject_id   hadm_id           admittime           dischtime deathtime  \
0    10467237  20000019 2159-03-20 21:08:00 2159-03-23 16:54:00       NaT   
1    16925328  20000024 2151-05-25 21:44:00 2151-05-26 18:36:00       NaT   
2    19430048  20000034 2174-05-22 19:13:00 2174-05-24 17:27:00       NaT   
3    18910522  20000041 2143-09-03 07:15:00 2143-09-06 13:50:00       NaT   
4    11146739  20000057 2190-01-15 17:07:00 2190-01-18 16:05:00       NaT   

                admission_type admit_provider_id  admission_location  \
0                     EW EMER.            P25U4M      EMERGENCY ROOM   
1                     EW EMER.            P54Q2B      EMERGENCY ROOM   
2           DIRECT OBSERVATION            P30SZF  PHYSICIAN REFERRAL   
3  SURGICAL SAME DAY ADMISSION            P48VXK  PHYSICIAN REFERRAL   
4               EU OBSERVATION            P77SO2      EMERGENCY ROOM   

         discharge_location insurance language marital_status  \
0                      HOME     Other  

In [16]:
ddf_adm_pat_merged['median_anchor_year'] = ddf_adm_pat_merged['anchor_year_group'].str.split('-').apply(lambda x: int((int(x[0]) + int(x[1])) / 2))
ddf_adm_pat_merged['admittime_year'] = ddf_adm_pat_merged['admittime'].dt.year
ddf_adm_pat_merged['diff_year'] = ddf_adm_pat_merged['anchor_year'] - ddf_adm_pat_merged['median_anchor_year']
ddf_adm_pat_merged['year_of_visit'] = ddf_adm_pat_merged['admittime_year'] - ddf_adm_pat_merged['diff_year']
ddf_adm_pat_merged['month_of_visit'] = ddf_adm_pat_merged['admittime'].dt.month
ddf_adm_pat_merged['day_of_visit'] = ddf_adm_pat_merged['admittime'].dt.day
ddf_adm_pat_merged['day_of_visit'] = ddf_adm_pat_merged.apply(
    lambda row: min(row['day_of_visit'], pd.Period(year=row['year_of_visit'], month=row['month_of_visit'], freq='M').days_in_month),
    axis=1
)

ddf_adm_pat_merged['visit_start_date'] = pd.to_datetime(
    {
        'year': ddf_adm_pat_merged['year_of_visit'],
        'month': ddf_adm_pat_merged['month_of_visit'],
        'day': ddf_adm_pat_merged['day_of_visit']
    }
)

print(ddf_adm_pat_merged[['visit_start_date']])

       visit_start_date
0            2012-03-20
1            2017-05-25
2            2020-05-22
3            2013-09-03
4            2014-01-15
...                 ...
431226       2009-08-09
431227       2009-07-05
431228       2015-09-24
431229       2015-10-27
431230       2019-07-28

[431231 rows x 1 columns]


In [17]:
ddf_adm_pat_merged['time_of_visit'] = ddf_adm_pat_merged['admittime'].dt.time
ddf_adm_pat_merged['visit_start_datetime'] = pd.to_datetime(
    {
        'year': ddf_adm_pat_merged['year_of_visit'],
        'month': ddf_adm_pat_merged['month_of_visit'],
        'day': ddf_adm_pat_merged['day_of_visit'],
        'hour': ddf_adm_pat_merged['time_of_visit'].apply(lambda x: x.hour),
        'minute': ddf_adm_pat_merged['time_of_visit'].apply(lambda x: x.minute),
        'second': ddf_adm_pat_merged['time_of_visit'].apply(lambda x: x.second)
    }
)

print(ddf_adm_pat_merged[['visit_start_datetime']])

       visit_start_datetime
0       2012-03-20 21:08:00
1       2017-05-25 21:44:00
2       2020-05-22 19:13:00
3       2013-09-03 07:15:00
4       2014-01-15 17:07:00
...                     ...
431226  2009-08-09 18:56:00
431227  2009-07-05 22:24:00
431228  2015-09-24 21:27:00
431229  2015-10-27 07:15:00
431230  2019-07-28 17:22:00

[431231 rows x 1 columns]


In [18]:
ddf_adm_pat_merged['discharged_year'] = ddf_adm_pat_merged['dischtime'].dt.year
ddf_adm_pat_merged['year_of_visit_end'] = ddf_adm_pat_merged['discharged_year'] - ddf_adm_pat_merged['diff_year']
ddf_adm_pat_merged['month_of_visit_end'] = ddf_adm_pat_merged['dischtime'].dt.month
ddf_adm_pat_merged['day_of_visit_end'] = ddf_adm_pat_merged['dischtime'].dt.day
ddf_adm_pat_merged['day_of_visit_end'] = ddf_adm_pat_merged.apply(
    lambda row: min(row['day_of_visit_end'], pd.Period(year=row['year_of_visit_end'], month=row['month_of_visit_end'], freq='M').days_in_month),
    axis=1
)
ddf_adm_pat_merged['visit_end_date'] = pd.to_datetime(
    {
        'year': ddf_adm_pat_merged['year_of_visit_end'],
        'month': ddf_adm_pat_merged['month_of_visit_end'],
        'day': ddf_adm_pat_merged['day_of_visit_end']
    }
)
print(ddf_adm_pat_merged[['visit_end_date']])

       visit_end_date
0          2012-03-23
1          2017-05-26
2          2020-05-24
3          2013-09-06
4          2014-01-18
...               ...
431226     2009-08-10
431227     2009-07-12
431228     2015-09-30
431229     2015-10-30
431230     2019-07-31

[431231 rows x 1 columns]


In [19]:
ddf_adm_pat_merged['time_of_visit_end'] = ddf_adm_pat_merged['dischtime'].dt.time
ddf_adm_pat_merged['visit_end_datetime'] = pd.to_datetime(
    {
        'year': ddf_adm_pat_merged['year_of_visit_end'],
        'month': ddf_adm_pat_merged['month_of_visit_end'],
        'day': ddf_adm_pat_merged['day_of_visit_end'],
        'hour': ddf_adm_pat_merged['time_of_visit_end'].apply(lambda x: x.hour),
        'minute': ddf_adm_pat_merged['time_of_visit_end'].apply(lambda x: x.minute),
        'second': ddf_adm_pat_merged['time_of_visit_end'].apply(lambda x: x.second)
    }
)

print(ddf_adm_pat_merged[['visit_end_datetime']])

        visit_end_datetime
0      2012-03-23 16:54:00
1      2017-05-26 18:36:00
2      2020-05-24 17:27:00
3      2013-09-06 13:50:00
4      2014-01-18 16:05:00
...                    ...
431226 2009-08-10 18:30:00
431227 2009-07-12 12:50:00
431228 2015-09-30 17:04:00
431229 2015-10-30 17:25:00
431230 2019-07-31 11:53:00

[431231 rows x 1 columns]


In [20]:
null_or_nat_count = ddf_adm_pat_merged['admittime'].isna().sum()
print("Number of null or NaT values in the 'admittime' column:", null_or_nat_count)
null_or_nat_count = ddf_adm_pat_merged['dischtime'].isna().sum()
print("Number of null or NaT values in the 'dischtime' column:", null_or_nat_count)

Number of null or NaT values in the 'admittime' column: 0
Number of null or NaT values in the 'dischtime' column: 0


In [21]:
null_or_nat_count = ddf_adm_pat_merged['visit_start_datetime'].isna().sum()
print("Number of null or NaT values in the 'visit_start_datetime' column:", null_or_nat_count)
null_or_nat_count = ddf_adm_pat_merged['visit_end_datetime'].isna().sum()
print("Number of null or NaT values in the 'visit_end_datetime' column:", null_or_nat_count)

Number of null or NaT values in the 'visit_start_datetime' column: 0
Number of null or NaT values in the 'visit_end_datetime' column: 0


In [22]:
ddf_adm_pat_merged['visit_type_concept_id'] = 32817
print(ddf_adm_pat_merged[['visit_type_concept_id']])

        visit_type_concept_id
0                       32817
1                       32817
2                       32817
3                       32817
4                       32817
...                       ...
431226                  32817
431227                  32817
431228                  32817
431229                  32817
431230                  32817

[431231 rows x 1 columns]


In [23]:
default_provider_id = 1
ddf_adm_pat_merged['provider_id'] = default_provider_id
print(ddf_adm_pat_merged[['provider_id']])

        provider_id
0                 1
1                 1
2                 1
3                 1
4                 1
...             ...
431226            1
431227            1
431228            1
431229            1
431230            1

[431231 rows x 1 columns]


In [24]:
ddf_adm_pat_merged['care_site_id'] = np.nan
print(ddf_adm_pat_merged[['care_site_id']])

        care_site_id
0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
...              ...
431226           NaN
431227           NaN
431228           NaN
431229           NaN
431230           NaN

[431231 rows x 1 columns]


In [25]:
ddf_adm_pat_merged['visit_source_concept_id'] = np.nan
print(ddf_adm_pat_merged[['visit_source_concept_id']])

        visit_source_concept_id
0                           NaN
1                           NaN
2                           NaN
3                           NaN
4                           NaN
...                         ...
431226                      NaN
431227                      NaN
431228                      NaN
431229                      NaN
431230                      NaN

[431231 rows x 1 columns]


In [26]:
ddf_adm_pat_merged['preceding_visit_occurrence_id'] = np.nan
print(ddf_adm_pat_merged[['preceding_visit_occurrence_id']])

        preceding_visit_occurrence_id
0                                 NaN
1                                 NaN
2                                 NaN
3                                 NaN
4                                 NaN
...                               ...
431226                            NaN
431227                            NaN
431228                            NaN
431229                            NaN
431230                            NaN

[431231 rows x 1 columns]


In [27]:
admission_location_mapping = {
    "EMERGENCY ROOM": 9203,
    "PHYSICIAN REFERRAL": None,
    "TRANSFER FROM HOSPITAL": 38004515,
    "WALK-IN/SELF REFERRAL": 38003620,
    "PROCEDURE SITE": None,
    "CLINIC REFERRAL": 8716,
    "PACU": None,
    "TRANSFER FROM SKILLED NURSING FACILITY": 8863,
    "INFORMATION NOT AVAILABLE": None,
    "INTERNAL TRANSFER TO OR FROM PSYCH": 38004284,
    "AMBULATORY SURGERY TRANSFER": 38004207,
}

ddf_adm_pat_merged['admitted_from_concept_id'] = ddf_adm_pat_merged['admission_location'].map(admission_location_mapping)
ddf_adm_pat_merged['admitted_from_concept_id'] = ddf_adm_pat_merged['admitted_from_concept_id'].astype('Int64')

print(ddf_adm_pat_merged[['admission_location', 'admitted_from_concept_id']])


        admission_location  admitted_from_concept_id
0           EMERGENCY ROOM                      9203
1           EMERGENCY ROOM                      9203
2       PHYSICIAN REFERRAL                      <NA>
3       PHYSICIAN REFERRAL                      <NA>
4           EMERGENCY ROOM                      9203
...                    ...                       ...
431226      EMERGENCY ROOM                      9203
431227      EMERGENCY ROOM                      9203
431228      EMERGENCY ROOM                      9203
431229  PHYSICIAN REFERRAL                      <NA>
431230  PHYSICIAN REFERRAL                      <NA>

[431231 rows x 2 columns]


In [28]:
discharge_location_mapping = {
    "HOME": 0,
    "HOME HEALTH CARE": 38004519,
    "SKILLED NURSING FACILITY": 8863,
    "DIED": None,
    "PSYCH FACILITY": 38004284,
    "REHAB": 38004526,
    "CHRONIC/LONG TERM ACUTE CARE": 38004277,
    "HOSPICE": 8546,
    "AGAINST ADVICE": None,
    "OTHER FACILITY": None,
    "ACUTE HOSPITAL": 38004279,
    "ASSISTED LIVING": 8615,
    "HEALTHCARE FACILITY": None,
}

ddf_adm_pat_merged['discharged_to_concept_id'] = ddf_adm_pat_merged['discharge_location'].map(discharge_location_mapping)
ddf_adm_pat_merged['discharged_to_concept_id'] = ddf_adm_pat_merged['discharged_to_concept_id'].astype('Int64')

print(ddf_adm_pat_merged[['discharge_location', 'discharged_to_concept_id']])


              discharge_location  discharged_to_concept_id
0                           HOME                         0
1               HOME HEALTH CARE                  38004519
2                           <NA>                      <NA>
3       SKILLED NURSING FACILITY                      8863
4                           <NA>                      <NA>
...                          ...                       ...
431226                      HOME                         0
431227                      HOME                         0
431228                      HOME                         0
431229                      HOME                         0
431230                      <NA>                      <NA>

[431231 rows x 2 columns]


In [29]:
admission_type_mapping = {
    "AMBULATORY OBSERVATION": 38004207,
    None: 9201,
    "DIRECT EMER.": 9203,
    "DIRECT OBSERVATION": 581385,
    "ELECTIVE": 9201,
    "EU OBSERVATION": 9203,
    "EW EMER.": 9203,
    "OBSERVATION ADMIT": 581385,
    "SURGICAL SAME DAY ADMISSION": 8883,
    "URGENT": 9203, 
}

ddf_adm_pat_merged['visit_concept_id'] = ddf_adm_pat_merged['admission_type'].map(admission_type_mapping)
ddf_adm_pat_merged['visit_concept_id'] = ddf_adm_pat_merged['visit_concept_id'].astype('Int64')

print(ddf_adm_pat_merged[['admission_type', 'visit_concept_id']])


                     admission_type  visit_concept_id
0                          EW EMER.              9203
1                          EW EMER.              9203
2                DIRECT OBSERVATION            581385
3       SURGICAL SAME DAY ADMISSION              8883
4                    EU OBSERVATION              9203
...                             ...               ...
431226                     EW EMER.              9203
431227                     EW EMER.              9203
431228                     EW EMER.              9203
431229  SURGICAL SAME DAY ADMISSION              8883
431230           DIRECT OBSERVATION            581385

[431231 rows x 2 columns]


In [30]:
ddf_adm_pat_merged_copy =  ddf_adm_pat_merged.copy()
ddf_adm_pat_merged_copy

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,time_of_visit_end,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_concept_id,preceding_visit_occurrence_id,admitted_from_concept_id,discharged_to_concept_id,visit_concept_id
0,10467237,20000019,2159-03-20 21:08:00,2159-03-23 16:54:00,NaT,EW EMER.,P25U4M,EMERGENCY ROOM,HOME,Other,...,16:54:00,2012-03-23 16:54:00,32817,1,NaN,NaN,NaN,9203,0,9203
1,16925328,20000024,2151-05-25 21:44:00,2151-05-26 18:36:00,NaT,EW EMER.,P54Q2B,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,...,18:36:00,2017-05-26 18:36:00,32817,1,NaN,NaN,NaN,9203,38004519,9203
2,19430048,20000034,2174-05-22 19:13:00,2174-05-24 17:27:00,NaT,DIRECT OBSERVATION,P30SZF,PHYSICIAN REFERRAL,<NA>,Medicare,...,17:27:00,2020-05-24 17:27:00,32817,1,NaN,NaN,NaN,<NA>,<NA>,581385
3,18910522,20000041,2143-09-03 07:15:00,2143-09-06 13:50:00,NaT,SURGICAL SAME DAY ADMISSION,P48VXK,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Other,...,13:50:00,2013-09-06 13:50:00,32817,1,NaN,NaN,NaN,<NA>,8863,8883
4,11146739,20000057,2190-01-15 17:07:00,2190-01-18 16:05:00,NaT,EU OBSERVATION,P77SO2,EMERGENCY ROOM,<NA>,Medicare,...,16:05:00,2014-01-18 16:05:00,32817,1,NaN,NaN,NaN,9203,<NA>,9203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,10382924,29999723,2170-08-09 18:56:00,2170-08-10 18:30:00,NaT,EW EMER.,P886Y0,EMERGENCY ROOM,HOME,Other,...,18:30:00,2009-08-10 18:30:00,32817,1,NaN,NaN,NaN,9203,0,9203
431227,11326722,29999745,2160-07-05 22:24:00,2160-07-12 12:50:00,NaT,EW EMER.,P64BMC,EMERGENCY ROOM,HOME,Other,...,12:50:00,2009-07-12 12:50:00,32817,1,NaN,NaN,NaN,9203,0,9203
431228,12133002,29999809,2172-09-24 21:27:00,2172-09-30 17:04:00,NaT,EW EMER.,P14D72,EMERGENCY ROOM,HOME,Medicare,...,17:04:00,2015-09-30 17:04:00,32817,1,NaN,NaN,NaN,9203,0,9203
431229,16572164,29999828,2180-10-27 07:15:00,2180-10-30 17:25:00,NaT,SURGICAL SAME DAY ADMISSION,P843YV,PHYSICIAN REFERRAL,HOME,Other,...,17:25:00,2015-10-30 17:25:00,32817,1,NaN,NaN,NaN,<NA>,0,8883


In [31]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')
omopcursor = conn.cursor()

In [32]:
visit_occurrence_data = ddf_adm_pat_merged_copy[['hadm_id', 'subject_id', 'visit_concept_id', 'visit_start_datetime', 'visit_end_datetime', 'visit_type_concept_id', 'provider_id', 'care_site_id', 'admission_type', 'visit_source_concept_id', 'admitted_from_concept_id', 'admission_location', 'discharged_to_concept_id', 'discharge_location', 'preceding_visit_occurrence_id']]

for index, row in visit_occurrence_data.iterrows():
    row = row.where(pd.notnull(row), None)
    
    sql = "INSERT INTO VISIT_OCCURRENCE (visit_occurrence_id, person_id, visit_concept_id, visit_start_datetime, visit_end_datetime, visit_type_concept_id, provider_id, care_site_id, visit_source_value, visit_source_concept_id, admitted_from_concept_id, admitted_from_source_value, discharge_to_concept_id, discharge_to_source_value, preceding_visit_occurrence_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    omopcursor.execute(sql, (
        row['hadm_id'],
        row['subject_id'],
        row['visit_concept_id'],
        row['visit_start_datetime'],
        row['visit_end_datetime'],
        row['visit_type_concept_id'],
        row['provider_id'],
        row['care_site_id'],
        row['admission_type'],
        row['visit_source_concept_id'],
        row['admitted_from_concept_id'],
        row['admission_location'],
        row['discharged_to_concept_id'],
        row['discharge_location'],
        row['preceding_visit_occurrence_id']
    ))

conn.commit()
print("Insertion was successful.")

Insertion was successful.
